In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
import pandas as pd
import os

In [ ]:
# Initialize the Chrome WebDriver
driver = webdriver.Chrome()

# Search query for Amazon
query = "laptop"

# File counter for saving HTML files
fileno = 0

# Loop through the first 4 pages of search results
for i in range(1, 5):
    # Construct and open the Amazon search results URL for the current page
    driver.get(f"https://www.amazon.com/s?k={query}&page={i}&crid=5XDWP0KYCLGQ&sprefix=laptop%2Caps%2C408&ref=nb_sb_ss_ts-doa-p_1_6")
    
    # Find all product elements on the page using the class name
    elems = driver.find_elements(By.CLASS_NAME, "puis-card-container")
    
    # Print the number of items found on the page
    print(f"{len(elems)} items found on page {i}")

    # Iterate over each product element
    for elem in elems:
        # Get the outer HTML of the element
        d = elem.get_attribute("outerHTML")
        
        # Save the HTML content to a file in the "data" directory
        with open(f"data/{query}_{fileno}.html", "w", encoding="utf-8") as f:
            f.write(d)
        
        # Increment file number for unique filenames
        fileno += 1

    # Pause for 2 seconds before loading the next page (to avoid being blocked)
    time.sleep(2)

# Close the browser after scraping is done
driver.close()


In [ ]:
# Initialize a dictionary to store extracted product details
d = {'title': [], 'price': [], 'link': []}

# Loop through all files in the "data" directory
for file in os.listdir("data"):
    try: 
        # Open and read the HTML file
        with open(f"data/{file}", "r", encoding="utf-8") as f:
            html_doc = f.read()

        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(html_doc, 'html.parser')

        # Extract the product title (assumed to be inside an <h2> tag)
        t = soup.find("h2")
        title = t.get_text()
    
        # Extract the product price (assumed to be inside a <span> tag with class "a-color-base")
        p = soup.find("span", attrs={"class": 'a-color-base'})
        price = p.get_text()
        
        # Extract the product link (finds the parent <a> tag of the title)
        l = t.find_parent('a')
        link = "https://www.amazon.com" + l.get('href')

        # Append extracted details to the dictionary
        d['title'].append(title)
        d['price'].append(price)
        d['link'].append(link)
        
        # Uncomment below if you want to see the extracted data while running
        # print(title, link, price)

    except Exception as e:
        # Print any errors encountered while processing a file
        print(e)

# Convert the dictionary into a DataFrame
df = pd.DataFrame(data=d)

# Save the extracted data into a CSV file
df.to_csv("data.csv", index=False)
